# Lamprey Transcriptome Analysis: Genome Completeness Assessment

```
Camille Scott [camille dot scott dot w @gmail.com] [@camille_codon]

camillescott.github.io

Lab for Data Intensive Biology (DIB)
University of California Davis
```

## About

Uses the many-sample de novo transcriptome assembly to attempt to assess the completeness of the genome build (and orthogonally, the completeness of the transcriptome itself).

    assembly version: lamp10

    assembly program: Trinity
    
    genome build: 7.0.75 (ensembl release 75

## Libraries

In [9]:
%load_ext autoreload
%autoreload 2
%autosave 60

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


Autosaving every 60 seconds


In [10]:
from libs import *
%run -i common.ipy

** Using data resources found in ../resources.json
** Using config found in ../config.json


In [11]:
import pyprind

In [13]:
from blasttools import blast_to_df

In [51]:
from IPython.display import display, Image
import glob

In [41]:
?Image

In [15]:
%pylab inline
from matplotlib import rc_context
tall_size = (8,16)
norm_size = (12,8)
mpl_params = {'figure.autolayout': True,
               'axes.titlesize': 24,
               'axes.labelsize': 16,
               'ytick.labelsize': 14,
               'xtick.labelsize': 14
               }
sns.set(style="white", palette="Paired", rc=mpl_params)
#sns.set_palette("Paired", desat=.6)
b, g, r, p = sns.color_palette("muted", 4)

Populating the interactive namespace from numpy and matplotlib


In [16]:
%config InlineBackend.close_figures = False

## Data

We'll save our results to a dictionary and dump it to JSON for use in the paper.

In [17]:
results = {}

In [18]:
store = pd.HDFStore(wdir('{}.store.h5'.format(prefix)), complib='zlib', complevel=5)

In [19]:
import atexit

In [20]:
def dump_results(fn='../doc/petmar-genome-comp.results.json'):
    with open(fn, 'wb') as fp:
        json.dump(results, fp)

In [26]:
store.close()

## Germline DNA Samples

We've got some nice shiny new sperm DNA samples - let's play with them some. First things first, we'll check out their FastQC results to make sure they're not crap.

In [49]:
fastqc_folders = sorted(glob.glob('*fastqc'))

In [109]:
!ls 2_ATCACG_L001_R1_001_fastqc/Images

duplication_levels.png	 per_base_sequence_content.png
kmer_profiles.png	 per_sequence_gc_content.png
per_base_gc_content.png  per_sequence_quality.png
per_base_n_content.png	 sequence_length_distribution.png
per_base_quality.png


In [ ]:
ls 

In [105]:
# quick utility function to generate a table of images
def get_img_table(folders, image):
    htmlout = '<table>'
    for i in xrange(0, len(folders), 2):
        left = fastqc_folders[i]
        right = fastqc_folders[i+1]
        htmlout += '<tr>'
        htmlout += '<td align="center">' + left + ', LEFT</td>'
        htmlout += '<td align="center">' + right + ', RIGHT</td>'
        htmlout += '</td>'
        
        htmlout += '<tr>'
        htmlout += '<td><img src="' + os.path.join(left, 'Images', image) + '"/></td>'
        htmlout += '<td><img src="' + os.path.join(right, 'Images', image) + '"/></td>'
        htmlout += '</tr>'
    htmlout += '</table>'
    return htmlout

#### Per-Base Sequence Quality

These look pretty normal -- in fact, the right fragments are a little better than I might expect for many runs.

In [113]:
display(HTML(get_img_table(fastqc_folders, 'per_base_quality.png')))

"2_ATCACG_L001_R1_001_fastqc, LEFT","2_ATCACG_L001_R2_001_fastqc, RIGHT"
,
"2_TGACCA_L001_R1_001_fastqc, LEFT","2_TGACCA_L001_R2_001_fastqc, RIGHT"
,
"4_CCGTCC_L001_R1_001_fastqc, LEFT","4_CCGTCC_L001_R2_001_fastqc, RIGHT"
,
"4_GATCAG_L001_R1_001_fastqc, LEFT","4_GATCAG_L001_R2_001_fastqc, RIGHT"
,


#### GC Content

For the most part, these results line up with what we expect from the lamprey genome paper: ~46% GC content for the whole genome (FastQC throws a warning for this check, but we can safely ignore it based on our prior knowledge). Curiously, in the genome paper, they report that coding regions had a GC content of ~61%, but we don't really see a bump in the distribution there -- instead, we see a bump at ~84% in all samples. There isn't anything unexpected in the per-*base* figures, so we can at least assume there isn't a technical artifact common to all reads.

In [110]:
display(HTML(get_img_table(fastqc_folders, 'per_sequence_gc_content.png')))

"2_ATCACG_L001_R1_001_fastqc, LEFT","2_ATCACG_L001_R2_001_fastqc, RIGHT"
,
"2_TGACCA_L001_R1_001_fastqc, LEFT","2_TGACCA_L001_R2_001_fastqc, RIGHT"
,
"4_CCGTCC_L001_R1_001_fastqc, LEFT","4_CCGTCC_L001_R2_001_fastqc, RIGHT"
,
"4_GATCAG_L001_R1_001_fastqc, LEFT","4_GATCAG_L001_R2_001_fastqc, RIGHT"
,


In [111]:
display(HTML(get_img_table(fastqc_folders, 'per_base_gc_content.png')))

"2_ATCACG_L001_R1_001_fastqc, LEFT","2_ATCACG_L001_R2_001_fastqc, RIGHT"
,
"2_TGACCA_L001_R1_001_fastqc, LEFT","2_TGACCA_L001_R2_001_fastqc, RIGHT"
,
"4_CCGTCC_L001_R1_001_fastqc, LEFT","4_CCGTCC_L001_R2_001_fastqc, RIGHT"
,
"4_GATCAG_L001_R1_001_fastqc, LEFT","4_GATCAG_L001_R2_001_fastqc, RIGHT"
,


Once more, nothing all too exciting -- a bit of A bias toward the end of the righ reads, which coincides with the expected drop in quality.

In [112]:
display(HTML(get_img_table(fastqc_folders, 'per_base_sequence_content.png')))

"2_ATCACG_L001_R1_001_fastqc, LEFT","2_ATCACG_L001_R2_001_fastqc, RIGHT"
,
"2_TGACCA_L001_R1_001_fastqc, LEFT","2_TGACCA_L001_R2_001_fastqc, RIGHT"
,
"4_CCGTCC_L001_R1_001_fastqc, LEFT","4_CCGTCC_L001_R2_001_fastqc, RIGHT"
,
"4_GATCAG_L001_R1_001_fastqc, LEFT","4_GATCAG_L001_R2_001_fastqc, RIGHT"
,


Seems a little odd that we get this slow increase in G/C homopolymers over the length of our reads. Need to investigate further.

In [114]:
display(HTML(get_img_table(fastqc_folders, 'kmer_profiles.png')))

"2_ATCACG_L001_R1_001_fastqc, LEFT","2_ATCACG_L001_R2_001_fastqc, RIGHT"
,
"2_TGACCA_L001_R1_001_fastqc, LEFT","2_TGACCA_L001_R2_001_fastqc, RIGHT"
,
"4_CCGTCC_L001_R1_001_fastqc, LEFT","4_CCGTCC_L001_R2_001_fastqc, RIGHT"
,
"4_GATCAG_L001_R1_001_fastqc, LEFT","4_GATCAG_L001_R2_001_fastqc, RIGHT"
,


In [115]:
display(HTML(get_img_table(fastqc_folders, 'duplication_levels.png')))

"2_ATCACG_L001_R1_001_fastqc, LEFT","2_ATCACG_L001_R2_001_fastqc, RIGHT"
,
"2_TGACCA_L001_R1_001_fastqc, LEFT","2_TGACCA_L001_R2_001_fastqc, RIGHT"
,
"4_CCGTCC_L001_R1_001_fastqc, LEFT","4_CCGTCC_L001_R2_001_fastqc, RIGHT"
,
"4_GATCAG_L001_R1_001_fastqc, LEFT","4_GATCAG_L001_R2_001_fastqc, RIGHT"
,


Summary: we should probably run a trimmer on these samples before using them. Shocker!